# GraphQL Multipart File Upload Returns 500 Error

This notebook demonstrates that the GraphQL `uploadFile` mutation returns a 500 error when using the standard GraphQL multipart request spec, while the REST `/images` endpoint works correctly.

## Environment
- Python 3.13
- requests library
- Valid OAuth2 token with `expenses` scope

In [1]:
import base64
import json
from io import BytesIO

import requests

# Create a minimal valid PNG (1x1 red pixel)
TEST_PNG = base64.b64decode(
    'iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mP8z8DwHwAFBQIAX8jx0gAAAABJRU5ErkJggg=='
)
print(f"Test PNG size: {len(TEST_PNG)} bytes")

Test PNG size: 70 bytes


In [2]:
# Load OAuth token (replace with your own or set ACCESS_TOKEN env var)
import os

ACCESS_TOKEN = os.environ.get('OPENCOLLECTIVE_ACCESS_TOKEN')
if not ACCESS_TOKEN:
    token_file = os.path.expanduser('~/.config/opencollective/token.json')
    if os.path.exists(token_file):
        with open(token_file) as f:
            ACCESS_TOKEN = json.load(f)['access_token']

print(f"Token loaded: {ACCESS_TOKEN[:20] if ACCESS_TOKEN else 'NOT FOUND'}...")

Token loaded: eyJhbGciOiJIUzI1NiIs...


## Verify Authentication Works

First, let's confirm the token is valid by querying the logged-in user:

In [3]:
API_URL = 'https://api.opencollective.com/graphql/v2'

# Simple query to verify auth
query = '{ me { id slug name } }'

response = requests.post(
    API_URL,
    json={'query': query},
    headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
)

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

Status: 200
Response: {
  "data": {
    "me": {
      "id": "x8k03rey-d5agmq5z-r8yqlbwo-z7j4nxv9",
      "slug": "max-ghenis",
      "name": "Max Ghenis"
    }
  }
}


## Verify Upload Scalar Exists

Check that the `Upload` scalar is defined in the schema:

In [4]:
query = '''{
  __type(name: "Upload") {
    name
    kind
    description
  }
}'''

response = requests.post(
    API_URL,
    json={'query': query},
    headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
)

print(f"Upload scalar exists: {response.json()}")

Upload scalar exists: {'data': {'__type': {'name': 'Upload', 'kind': 'SCALAR', 'description': 'The `Upload` scalar type represents a file upload.'}}}


## BUG: GraphQL Multipart Upload Returns 500

Following the [GraphQL multipart request spec](https://github.com/jaydenseric/graphql-multipart-request-spec), this should work but returns 500:

In [5]:
mutation = '''mutation UploadFile($files: [UploadFileInput!]!) {
  uploadFile(files: $files) {
    file {
      id
      url
      name
    }
  }
}'''

operations = {
    'query': mutation,
    'variables': {
        'files': [{'kind': 'EXPENSE_ATTACHED_FILE', 'file': None}]
    }
}

file_map = {'0': ['variables.files.0.file']}

files = {
    'operations': (None, json.dumps(operations), 'application/json'),
    'map': (None, json.dumps(file_map), 'application/json'),
    '0': ('test.png', BytesIO(TEST_PNG), 'image/png'),
}

response = requests.post(
    API_URL,
    files=files,
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'}
)

print(f"GraphQL multipart upload status: {response.status_code}")
print(f"Response: {response.text}")

GraphQL multipart upload status: 500
Response: {"error":{"code":500}}


## Same Request with Apollo-Require-Preflight Header

The frontend uses `Apollo-Require-Preflight: true` header. Let's try that:

In [6]:
files = {
    'operations': (None, json.dumps(operations), 'application/json'),
    'map': (None, json.dumps(file_map), 'application/json'),
    '0': ('test.png', BytesIO(TEST_PNG), 'image/png'),
}

response = requests.post(
    API_URL,
    files=files,
    headers={
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Apollo-Require-Preflight': 'true'
    }
)

print(f"With Apollo-Require-Preflight header status: {response.status_code}")
print(f"Response: {response.text}")

With Apollo-Require-Preflight header status: 500
Response: {"error":{"code":500}}


## WORKAROUND: REST /images Endpoint Works

The REST endpoint works correctly:

In [7]:
REST_URL = 'https://api.opencollective.com/images'

files = {
    'file': ('test.png', BytesIO(TEST_PNG), 'image/png'),
}

data = {
    'kind': 'EXPENSE_ATTACHED_FILE',
}

response = requests.post(
    REST_URL,
    files=files,
    data=data,
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'}
)

print(f"REST /images endpoint status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

REST /images endpoint status: 200
Response: {
  "status": 200,
  "url": "https://opencollective.com/api/files/a47byg9n-xozdp8y5-gr9qmjlv-03rek5w8"
}


## Summary

| Endpoint | Method | Status |
|----------|--------|--------|
| `/graphql/v2` | GraphQL multipart | **500 Error** |
| `/graphql/v2` | GraphQL multipart + Apollo header | **500 Error** |
| `/images` | REST multipart | **200 OK** |

The GraphQL `uploadFile` mutation is defined in the schema and the `Upload` scalar exists, but multipart uploads via the standard spec return 500 errors. The REST `/images` endpoint works as a workaround.